In [ ]:
from VR_Trajectory_analysis import *

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR3/20250522_VR3StaticDecisions_V0/20250605_VR3StaticDecisions_V1_Data/RunData'

In [ ]:
df = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
df["FlyID"].nunique()

In [ ]:
df.columns

In [ ]:
df = add_trial_id_and_displacement(df)
df = add_trial_time(df)

In [ ]:
df['trial_time'].describe()

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_path_length(df[(df['stepName'] != 'skybox') & (df['TotalDisplacement'] > 12)], min_length=0, max_length=100)

In [ ]:
inter_stationary, inter_normal, inter_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_path_length(df[(df['stepName'] == 'skybox')], min_length=3, max_length=20)

In [ ]:
inter_stationary['UniqueTrialID'].nunique(), inter_normal['UniqueTrialID'].nunique(), inter_excessive['UniqueTrialID'].nunique()

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(inter_stationary['TotalPathLength'], bins=30)
plt.title('Total displacement per trial (inter_normal)')
plt.xlabel('Total displacement (units)')        
plt.ylabel('Number of trials')

plt.tight_layout()
plt.show()

In [ ]:
plot_trajectories(inter_stationary, 'normal')

In [ ]:
plot_trajectories(inter_normal, 'normal')

In [ ]:
# 3. Calculate step-wise time difference (in seconds)
inter_normal['step_time'] = inter_normal.groupby('UniqueTrialID')['Current Time'].diff().dt.total_seconds()

# 4. Calculate step-wise velocity
inter_normal['step_velocity'] = inter_normal['step_distance'] / inter_normal['step_time']

# Replace infinite or NaN values with 0 (first row in each group will have NaN)
inter_normal['step_velocity'] = inter_normal['step_velocity'].replace([np.inf, -np.inf], np.nan).fillna(0)
inter_normal['smoothed_velocity'] = inter_normal['step_velocity'].rolling(window=7, center=True).mean()
# Make a copy to avoid modifying original
df_analysis_inter = inter_normal.copy()

In [ ]:
# 3. Calculate step-wise time difference (in seconds)
inter_stationary['step_time'] = inter_stationary.groupby('UniqueTrialID')['Current Time'].diff().dt.total_seconds()

# 4. Calculate step-wise velocity
inter_stationary['step_velocity'] = inter_stationary['step_distance'] / inter_stationary['step_time']

# Replace infinite or NaN values with 0 (first row in each group will have NaN)
inter_stationary['step_velocity'] = inter_stationary['step_velocity'].replace([np.inf, -np.inf], np.nan).fillna(0)
inter_stationary['smoothed_velocity'] = inter_stationary['step_velocity'].rolling(window=7, center=True).mean()

In [ ]:
# Group by UniqueTrialID and get mean smoothed_velocity and min elapsed_time for each trial
trial_velocity_summary = (
    df_analysis.groupby('UniqueTrialID')
    .agg(mean_smoothed_velocity=('smoothed_velocity', 'mean'),
         min_elapsed_time=('elapsed_time', 'min'))
    .reset_index()
)

# Plot: x = min_elapsed_time, y = mean_smoothed_velocity, one point per trial
plt.figure(figsize=(8, 5))
plt.scatter(trial_velocity_summary['min_elapsed_time'],
            trial_velocity_summary['mean_smoothed_velocity'],
            alpha=0.2)
plt.xlabel('Min Elapsed Time (s)')
plt.ylabel('Mean Smoothed Velocity')
plt.title('Mean Smoothed Velocity per Trial vs. Min Elapsed Time')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(inter_normal['smoothed_velocity'], bins=300, edgecolor='k', alpha=0.7)
plt.title('Histogram of Smoothed Velocity')
plt.xlabel('Smoothed Velocity')
plt.xlim(0, 10)  # Adjust x-axis range as needed
plt.ylabel('Number of Trials')
plt.tight_layout()
plt.show()
plt.figure(figsize=(6, 4))
plt.hist(inter_stationary['smoothed_velocity'], bins=300, edgecolor='k', alpha=0.7)
plt.title('Histogram of Smoothed Velocity')
plt.xlabel('Smoothed Velocity')
plt.xlim(0, 10)  # Adjust x-axis range as needed
plt.ylabel('Number of Trials')
plt.tight_layout()
plt.show()
plt.hist(df_normal['smoothed_velocity'], bins=300, edgecolor='k', alpha=0.7)
plt.title('Histogram of Smoothed Velocity')
plt.xlabel('Smoothed Velocity')
plt.xlim(0, 10)  # Adjust x-axis range as needed
plt.ylabel('Number of Trials')
plt.tight_layout()
plt.show()

In [ ]:
trial_durations = (
    df_normal.groupby('UniqueTrialID')['elapsed_time']
    .agg(TrialStart='min', TrialEnd='max')
    .assign(TrialDuration=lambda g: g['TrialEnd'] - g['TrialStart'])
    .reset_index()
)
long_trial_ids = trial_durations[trial_durations['TrialDuration'] > 8]['UniqueTrialID']
df_normal_long = df_normal[df_normal['UniqueTrialID'].isin(long_trial_ids)].copy()

# Optional: merge duration info into df_normal_long
df_normal_long = df_normal_long.merge(trial_durations[['UniqueTrialID', 'TrialDuration']], on='UniqueTrialID', how='left')


In [ ]:
import matplotlib.pyplot as plt

# one value per trial ─ choose either pattern
# -----------------------------------------------------------------
# 1) group-then-first  (safe even if some trials have NaNs)
trial_disp = (
    df_normal_long.groupby('UniqueTrialID')['TotalDisplacement']
             .first()              # first (== only) value for that trial
             .dropna()             # keep clean
)

# --- OR ----------------------------------------------------------
# 2) drop duplicates if every row already carries the same number
#    for that trial
# trial_disp = (
#     df_normal[['UniqueTrialID', 'TotalDisplacement']]
#       .drop_duplicates('UniqueTrialID')
#       .set_index('UniqueTrialID')['TotalDisplacement']
# )

# -----------------------------------------------------------------
# histogram
plt.figure(figsize=(6, 4))
plt.hist(trial_disp, bins=30)
plt.title('Total displacement per trial')
plt.xlabel('Total displacement (units)')
plt.ylabel('Number of trials')
plt.tight_layout()
plt.show()


In [ ]:
plot_trajectories(df_normal, 'normal')

In [ ]:
#add angle column to df_joined
df_normal_long['Angle'] = df_normal_long['stepName'].apply(lambda x: parse_angle_from_config(x, default_angle=999.0))

In [ ]:
df_normal= df_normal_long

In [ ]:
df_normal['FlyID'].nunique()

In [ ]:
# -----------------------------------------------------------------
#  30 random trajectories from df_normal, labelled by displacement
# -----------------------------------------------------------------
import math, numpy as np, matplotlib.pyplot as plt
from matplotlib import gridspec

# ------------------------ parameters ----------------------------
N_TRIALS   = 30          # how many to draw
SEED       = 42          # set None for a different selection every run
COLS       = 5           # grid width  (→ 6 rows for 30plots)

# ---------------------  draw the sample -------------------------
rng   = np.random.default_rng(SEED)
all_ids = df_normal['UniqueTrialID'].dropna().unique()
if len(all_ids) < N_TRIALS:
    raise ValueError(f"Only {len(all_ids)} trials in df_normal, need {N_TRIALS}.")

sample_ids = rng.choice(all_ids, size=N_TRIALS, replace=False)

# -----------------  prepare the figure grid ---------------------
ROWS  = math.ceil(N_TRIALS / COLS)
fig   = plt.figure(figsize=(COLS*4, ROWS*3.5))
gs    = gridspec.GridSpec(ROWS, COLS, wspace=.25, hspace=.35)

# ------------------  helper: displacement -----------------------
def displacement(trial_df) -> float:
    """Euclidean distance between first and last (X, Z) positions."""
    x0, z0 = trial_df[['GameObjectPosX', 'GameObjectPosZ']].iloc[0]
    x1, z1 = trial_df[['GameObjectPosX', 'GameObjectPosZ']].iloc[-1]
    return math.hypot(x1 - x0, z1 - z0)

# ------------------  plot each sampled trial --------------------
for idx, trial_id in enumerate(sample_ids):
    ax = fig.add_subplot(gs[idx])

    td = df_normal[df_normal['UniqueTrialID'] == trial_id].sort_values('elapsed_time')

    # trajectory
    ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], lw=1)

    # aesthetics
    ax.set_xlim(-70, 70);  ax.set_ylim(-70, 70)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xticks([]); ax.set_yticks([])      # hide ticks for clarity

    # title with displacement
    disp = displacement(td)
    ax.set_title(f"ID {trial_id}\nΔ = {disp:.1f} units", fontsize=9)

# hide any leftover empty cells
for j in range(idx+1, ROWS*COLS):
    fig.add_subplot(gs[j]).axis('off')

fig.suptitle("Random sample of 30 trajectories (labelled by displacement)",
             y=.92, fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------------
#  Trajectories for all  "BlueCylinder_BlueGreenCylinder" configs
#  shown in a grid, one panel per Angle
# -----------------------------------------------------------------
import math, re
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec

# 1.  The joined dataframe is assumed to exist:  df_normal



df_filt = df_normal

# make sure Angle is numeric so the sort works
df_filt['Angle'] = pd.to_numeric(df_filt['Angle'], errors='coerce')

angles = sorted(df_filt['Angle'].dropna().unique())
if not angles:
    raise ValueError("No matching configs with an Angle column found.")

# 3.  Build grid layout -----------------------------------------------------
n_panels = len(angles)
n_cols   = 3                               # change to taste
n_rows   = math.ceil(n_panels / n_cols)

fig = plt.figure(figsize=(n_cols*4, n_rows*3.75))
gs  = gridspec.GridSpec(n_rows, n_cols, wspace=.25, hspace=.35)

# 4.  One sub-plot per Angle -------------------------------------------------
for idx, angle in enumerate(angles):
    ax = fig.add_subplot(gs[idx])
    g  = df_filt[df_filt['Angle'] == angle]

    # trajectories
    for trial_id, td in g.groupby('UniqueTrialID'):
        ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], alpha=.1)

    # basic aesthetics
    ax.set_xlim(-40, 40)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')

    ax.set_xticks([-60, -40, -20, 0, 20, 40, 60])
    ax.set_yticks([0, 20, 40, 60])
    ax.tick_params(labelsize=8)

    ax.set_title(f"{int(angle)}°", fontsize=10)


# hide any leftover empty cells
for j in range(idx+1, n_rows*n_cols):
    fig.add_subplot(gs[j]).axis('off')

fig.suptitle("Trajectories – BlueCylinder_BlueGreenCylinder", y=.92, fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------------------------------------------
#  One sample trajectory  +  its orientation histogram  per Angle
# -----------------------------------------------------------------
import math
import matplotlib.pyplot as plt
from matplotlib import gridspec

# --- pick one exemplar trial-id for every angle ---------------------------
# (first ID is deterministic; swap `.iloc[0]` with `sample(1).iat[0]`
#  if you prefer a random example each run)
examples = (
    df_filt
    .groupby('Angle')['UniqueTrialID']
    .first()                       # or .sample(1).iat[0]
    .to_dict()                     # {angle : trial_id}
)

angles = sorted(examples)          # same ordering as before

# --- figure layout: 2 cols (traj | hist) ----------------------------------
n_rows  = len(angles)
fig     = plt.figure(figsize=(10, 3.5 * n_rows))
gs      = gridspec.GridSpec(n_rows, 2, wspace=.35, hspace=.4)

for r, angle in enumerate(angles):

    trial_id = examples[angle]
    td       = df_filt[df_filt['UniqueTrialID'] == trial_id]

    # ── (a) trajectory ----------------------------------------------------
    ax = fig.add_subplot(gs[r, 0])
    ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], lw=1)
    ax.set_xlim(-40, 40);  ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"Angle {int(angle)}° – trial {trial_id}", fontsize=10)
    ax.set_xlabel('X');  ax.set_ylabel('Z')

    # ── (b) histogram of orientations ------------------------------------
    axh = fig.add_subplot(gs[r, 1])
    axh.hist(td['GameObjectRotY'], bins=30, alpha=.8, edgecolor='k')
    axh.set_xlabel('Orientation Y (deg)')
    axh.set_ylabel('Count')
    axh.set_title('Orientation distribution', fontsize=10)
    # NEW – get the pre-computed overall travel direction for this trial
    travel_dir = df_filt.loc[
        df_filt['UniqueTrialID'] == trial_id, 'TravelDirectionDeg'
    ].iloc[0]

    # NEW – draw it as a vertical line on the histogram
    axh.axvline(travel_dir, color='red', lw=2, ls='--',
                label=f"travel dir ({travel_dir:.0f}°)")

    # show the label only once (first row) to avoid legend clutter
    if r == 0:
        axh.legend(frameon=False, fontsize=8)

fig.suptitle("Sample trajectory + orientation histogram per platform angle",
             y=.92, fontsize=15)
plt.tight_layout()
plt.show()


In [ ]:
# -------- 1. summarise to one row per trial -------------------------------
trial_summary = (
    df_normal
      .sort_values(['UniqueTrialID', 'elapsed_time'])
      .groupby('UniqueTrialID')
      .agg(angle=('Angle', 'first'),
           travel_dir=('TravelDirectionDeg', 'first'))
      .dropna(subset=['angle', 'travel_dir'])
      .reset_index()
)

# convert Unity heading (0–360) → signed heading (-180…+180)  ⟹  0 in the middle
trial_summary['travel_dir_signed'] = (
    (trial_summary['travel_dir'] + 180) % 360    # shift, wrap into 0-360
    - 180                                        # back to signed range
)

# -------- 2. plot ----------------------------------------------------------
plt.figure(figsize=(6, 6))
plt.scatter(trial_summary['angle'],
            trial_summary['travel_dir_signed'],
            s=25, alpha=.2, color='black')

# -----------------------------------------------------------------
# NEW: goal-direction reference lines  (  y = ±½x  )
# -----------------------------------------------------------------
x_line = np.linspace(trial_summary['angle'].min() - 5,
                     trial_summary['angle'].max() + 5, 200)
plt.plot(x_line,  0.5 * x_line, ls='--', c='tab:red',  lw=0.5)
plt.plot(x_line, -0.5 * x_line, ls='--', c='tab:red', lw=0.5)

plt.xlabel('conflict angle (deg)')
plt.ylabel('Travel direction (deg, signed)')
plt.title('overall heading vs. conflict angle\n(one point per trial)')

plt.xlim(trial_summary['angle'].min() - 5,
         trial_summary['angle'].max() + 5)
plt.ylim(-180, 180)
plt.yticks(range(-180, 181, 60))
plt.axhline(0, color='grey', ls='--', lw=0.8)
plt.legend(frameon=False, fontsize=8)      # optional
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Prepare data
x = np.abs(trial_summary['angle'].values)
y = np.abs(trial_summary['travel_dir_signed'].values)

# Piecewise model: constant 'a' below x_c, slope 'b' above
def piecewise_model(x, a, b, x_c):
    return np.where(x < x_c, a, b * x)

# Loss function: mean squared error between model and data
def loss(params):
    a, b, x_c = params
    y_pred = piecewise_model(x, a, b, x_c)
    return np.mean((y - y_pred) ** 2)

# Initial guess: [a, b, x_c]
initial_guess = [5, 0.5, 60]

# Bounds to keep things realistic
bounds = [(0, 30),      # a: subcritical baseline
          (0.1, 2),     # b: supercritical slope
          (0, max(x))]  # x_c: critical angle

# Fit the model
result = minimize(loss, x0=initial_guess, bounds=bounds)
a_fit, b_fit, xc_fit = result.x

print(f"Fitted parameters:")
print(f"  a (subcritical baseline): {a_fit:.2f}")
print(f"  b (supercritical slope):  {b_fit:.2f}")
print(f"  x_c (critical angle):     {xc_fit:.2f}°")

# Plot
x_fit = np.linspace(min(x), max(x), 300)
y_fit = piecewise_model(x_fit, a_fit, b_fit, xc_fit)

plt.figure(figsize=(8, 6))
plt.scatter(x, y, alpha=0.1, color='black', label='Data')
plt.plot(x_fit, y_fit, color='tab:blue', label='Piecewise fit')
plt.axvline(xc_fit, color='tab:red', ls='--', label=f'Critical angle ≈ {xc_fit:.1f}°')
plt.xlabel('ABS Conflict angle (deg)')
plt.ylabel('ABS Travel direction (deg)')
plt.title('Piecewise Fit: Flat below, Linear above Critical Angle')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure 'Current Time' is in datetime format
df_normal['Current Time'] = pd.to_datetime(df_normal['Current Time'])

# Preprocess: relative time per trial
df_normal['Relative Time'] = df_normal.groupby('UniqueTrialID')['Current Time'].transform(
    lambda x: (x - x.min()).dt.total_seconds()
)

# Sort just to be safe
df_normal = df_normal.sort_values(['Angle', 'UniqueTrialID', 'Relative Time'])

# Function to insert NaNs where centered angle jumps > threshold
def break_large_centered_jumps(group, threshold=300):
    # Convert to centered angles: [-180, 180)
    angle_centered = ((group['GameObjectRotY'] + 180) % 360) - 180
    diff = angle_centered.diff().abs()

    # Break jumps
    angle_broken = angle_centered.copy()
    angle_broken[diff > threshold] = np.nan

    group['AnglePlotCentered'] = angle_broken
    return group
df_normal = df_normal.groupby('UniqueTrialID', group_keys=False).apply(break_large_centered_jumps)

# Get all unique angles
unique_angles = sorted(df_normal['Angle'].dropna().unique())
n_angles = len(unique_angles)

# Create subplots
fig, axes = plt.subplots(n_angles, 1, figsize=(12, 4 * n_angles), sharex=False)

if n_angles == 1:
    axes = [axes]  # Ensure axes is iterable

for ax, angle in zip(axes, unique_angles):
    df_angle = df_normal[df_normal['Angle'] == angle]
    for trial_id in df_angle['UniqueTrialID'].unique():
        trial_data = df_angle[df_angle['UniqueTrialID'] == trial_id]
        ax.plot(trial_data['Relative Time'], trial_data['AnglePlotCentered'], label=f'Trial {trial_id}')
    
    ax.set_title(f'Angle = {angle}')
    ax.set_ylabel('Travel Direction (Degrees)')
    ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))

axes[-1].set_xlabel('Relative Time (seconds)')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Create subplots
fig, axes = plt.subplots(n_angles, 1, figsize=(12, 4 * n_angles), sharex=False)
if n_angles == 1:
    axes = [axes]  # Ensure iterable

for ax, angle in zip(axes, unique_angles):
    df_angle = df_normal[df_normal['Angle'] == angle]
    
    # Map UniqueFlyID to colors
    unique_flies = df_angle['FlyID'].unique()
    cmap = cm.get_cmap('tab20', len(unique_flies))  # Use tab20 or another suitable colormap
    fly_color_map = {fly_id: cmap(i) for i, fly_id in enumerate(unique_flies)}
    
    for trial_id in df_angle['UniqueTrialID'].unique():
        trial_data = df_angle[df_angle['UniqueTrialID'] == trial_id]
        fly_id = trial_data['FlyID'].iloc[0]  # Get fly ID for this trial
        color = fly_color_map.get(fly_id, 'gray')   # fallback color just in case
        ax.plot(trial_data['Relative Time'], trial_data['AnglePlotCentered'],
                label=f'Fly {fly_id}', color=color, alpha=0.8)

    ax.set_title(f'Angle = {angle}')
    ax.set_ylabel('Travel Direction (Degrees)')
    
    # Optional: remove repeated labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right', bbox_to_anchor=(1.15, 1), fontsize=8)

axes[-1].set_xlabel('Relative Time (seconds)')
plt.tight_layout()
plt.show()


In [ ]:
# 3. Calculate step-wise time difference (in seconds)
df_normal['step_time'] = df_normal.groupby('UniqueTrialID')['Current Time'].diff().dt.total_seconds()

# 4. Calculate step-wise velocity
df_normal['step_velocity'] = df_normal['step_distance'] / df_normal['step_time']

# Replace infinite or NaN values with 0 (first row in each group will have NaN)
df_normal['step_velocity'] = df_normal['step_velocity'].replace([np.inf, -np.inf], np.nan).fillna(0)
df_normal['smoothed_velocity'] = df_normal['step_velocity'].rolling(window=7, center=True).mean()
# Make a copy to avoid modifying original
df_analysis = df_normal.copy()

# Smooth the velocity per trial (7-step window, centered)
df_analysis['smoothed_velocity'] = (
    df_analysis.groupby('UniqueTrialID')['step_velocity']
    .transform(lambda x: x.rolling(window=7, center=True, min_periods=1).mean())
)

# Define relative time bins — you can tweak the bin size
bin_size = 0.25  # seconds
max_relative_time = df_analysis['Relative Time'].max()
bins = np.arange(0, max_relative_time + bin_size, bin_size)

# Assign each row to a time bin
df_analysis['time_bin'] = pd.cut(df_analysis['Relative Time'], bins=bins, right=False)

# Group by time bin and compute mean and SEM (optional) across all trials
summary = (
    df_analysis.groupby('time_bin')['smoothed_velocity']
    .agg(['mean', 'count', 'std'])
    .rename(columns={'mean': 'mean_velocity', 'std': 'std_velocity'})
)
summary['sem_velocity'] = summary['std_velocity'] / np.sqrt(summary['count'])

# Midpoint of each bin for plotting
summary['bin_center'] = [interval.left + bin_size / 2 for interval in summary.index]


In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(summary['bin_center'], summary['mean_velocity'], label='Mean Velocity')
plt.fill_between(summary['bin_center'],
                 summary['mean_velocity'] - summary['sem_velocity'],
                 summary['mean_velocity'] + summary['sem_velocity'],
                 alpha=0.3, label='±1 SEM')
plt.xlabel('Relative Time (s)')
plt.ylabel('Mean Smoothed Velocity')
plt.title('Mean Smoothed Velocity Across All Trials Over Time')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Convert degrees to radians
theta_rad = np.deg2rad(df_analysis['GameObjectRotY'])

# Compute angle difference in radians (handles wrapping properly)
turn_rate_rad = np.angle(np.exp(1j * np.diff(theta_rad, prepend=np.nan)))

# Convert to degrees
df_analysis['turning_rate_deg'] = np.rad2deg(turn_rate_rad)

# Absolute turning rate (how much turning, regardless of direction)
df_analysis['abs_turning_rate'] = np.abs(df_analysis['turning_rate_deg'])

df_analysis['turning_rate_deg_per_sec'] = (
    df_analysis['turning_rate_deg'] / df_analysis['step_time']
).replace([np.inf, -np.inf], np.nan).fillna(0)

df_analysis['abs_turning_rate_per_sec'] = np.abs(df_analysis['turning_rate_deg_per_sec'])

# Compute mean turning rate per trial from the raw dataframe
trial_turning = (
    df_analysis
    .groupby('UniqueTrialID')['abs_turning_rate_per_sec']
    .mean()
    .reset_index()
    .rename(columns={'abs_turning_rate_per_sec': 'mean_turning_rate'})
)


# Get one Angle per trial
trial_angles = (
    df_analysis
    .groupby('UniqueTrialID')['Angle']
    .first()
    .reset_index()
)

# Merge it in
trial_turning = trial_turning.merge(trial_angles, on='UniqueTrialID')


angle_summary_raw = (
    trial_turning
    .groupby('Angle')['mean_turning_rate']
    .agg(['mean', 'std', 'count'])
    .reset_index()
)
angle_summary_raw['sem'] = angle_summary_raw['std'] / np.sqrt(angle_summary_raw['count'])

angle_summary_raw = (
    trial_turning
    .groupby('Angle')['mean_turning_rate']
    .agg(['mean', 'std', 'count'])
    .reset_index()
)
angle_summary_raw['sem'] = angle_summary_raw['std'] / np.sqrt(angle_summary_raw['count'])

plt.figure(figsize=(12, 5))
plt.errorbar(angle_summary_raw['Angle'], angle_summary_raw['mean'], yerr=angle_summary_raw['sem'],
             fmt='o', ecolor='gray', elinewidth=1, capsize=2, alpha=0.8)
plt.xlabel('Angle (degrees)')
plt.ylabel('Mean Turning Rate (°/s)')
plt.title('Mean Turning Rate by Raw Angle')
plt.grid(True)
plt.tight_layout()
plt.show()
